In [2]:
# %% [Section 1] 라이브러리 및 데이터 로드
import pandas as pd
import numpy as np
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, label_binarize
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import warnings

warnings.filterwarnings('ignore')

# 1. 데이터 로드 (경로 수정 필요)
train_path = r"C:\Users\abc01\OneDrive\바탕 화면\train.csv"
test_path  = r"C:\Users\abc01\OneDrive\바탕 화면\test.csv"

train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)

print(f"Original Train shape: {train_df.shape}")
print(f"Original Test shape:  {test_df.shape}")

Original Train shape: (6500, 24)
Original Test shape:  (1405, 23)


In [4]:
# %% [Section 2] 피처 엔지니어링 함수 정의

# 1. EDA Rule Features (Int형과 String형 둘 다 생성)
def add_eda_rule_features_v2(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) chem_01_range
    df['chem_01_range_str'] = pd.cut(
        df['chem_01'], 
        bins=[-np.inf, 2.0, 4.0, np.inf], 
        labels=['Normal_Low', 'Warning_Zone', 'Danger_High']
    ).astype(str)

    chem_map = {'Normal_Low': 0, 'Warning_Zone': 1, 'Danger_High': 2}
    df['chem_01_range'] = df['chem_01_range_str'].map(chem_map).astype(int)

    # 2) trace_metal_range
    df['trace_metal_range_str'] = pd.cut(
        df['trace_metal'],
        bins=[-np.inf, 90, 130, np.inf],
        labels=['Safe', 'Caution', 'Risk']
    ).astype(str)

    metal_map = {'Safe': 0, 'Caution': 1, 'Risk': 2}
    df['trace_metal_range'] = df['trace_metal_range_str'].map(metal_map).astype(int)

    # 3) risk_segment
    def check_ambiguous_risk(row):
        if (row['swelling'] == 'S') and (2.0 <= row['chem_01'] < 5.0):
            return 'Target_1_Suspect'
        elif (row['swelling'] == 'Y') or (row['chem_01'] >= 5.0):
            return 'High_Risk'
        else:
            return 'Normal'

    df['risk_segment_str'] = df.apply(check_ambiguous_risk, axis=1).astype(str)
    risk_map = {'Normal': 0, 'Target_1_Suspect': 1, 'High_Risk': 2}
    df['risk_segment'] = df['risk_segment_str'].map(risk_map).astype(int)

    return df

# 2. Boosting Features (행 단위 연산)
def make_boosting_features_v2(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 기본 수치형 파생
    high_is_healthy = ["protein_level", "blood_cells"]
    low_is_healthy  = ["enzyme_A", "enzyme_B", "lipid_index", "trace_metal", "clot_time"]

    def minmax(s):
        s = s.astype(float)
        mn, mx = s.min(), s.max()
        if mx == mn: return pd.Series(0.5, index=s.index)
        return (s - mn) / (mx - mn + 1e-6)

    if all(col in df.columns for col in high_is_healthy + low_is_healthy):
        df["health_index"] = (
            sum(minmax(df[col]) for col in high_is_healthy)
            - sum(minmax(df[col]) for col in low_is_healthy)
        )
        df["health_per_stage"] = df["health_index"] / (df["disease_stage"] + 1e-6)

    # 상호작용 / 비율 / 합·차
    df["chem01_trace_combo"]   = df["chem_01"] * df["trace_metal"]
    df["chem01_chem02_combo"]  = df["chem_01"] * df["chem_02"]
    df["chem01_enzymeB_combo"] = df["chem_01"] * df["enzyme_B"]
    df["enzyme_ratio"]       = df["enzyme_A"] / (df["enzyme_B"].replace(0, np.nan) + 1e-6)
    df["lipid_blood_ratio"]  = df["lipid_index"] / (df["blood_cells"].replace(0, np.nan) + 1e-6)
    df["chem02_trace_ratio"] = df["chem_02"] / (df["trace_metal"].replace(0, np.nan) + 1e-6)
    
    df["sum_chem"]   = df["chem_01"] + df["chem_02"]
    df["diff_chem"]  = df["chem_01"] - df["chem_02"]
    df["sum_enzyme"] = df["enzyme_A"] + df["enzyme_B"]
    df["diff_enzyme"] = df["enzyme_A"] - df["enzyme_B"]

    # 시간/나이 정규화
    if "age" in df.columns:
        df["obs_per_age"]      = df["obs_days"] / (df["age"] + 1e-6)
        df["behavior_per_age"] = df["behavior_index"] / (df["age"] + 1e-6)
    df["disease_velocity"] = df["disease_stage"] / (df["obs_days"] + 1e-6)

    # 증상 카운트 & Swelling 변환
    symptom_cols = ["fluid_accum", "organ_enlarge", "vascular_marks"]
    # replace 시 안전하게 처리
    temp_symptoms = df[symptom_cols].replace({"Y": 1, "N": 0}).infer_objects(copy=False)
    df["symptom_count"] = temp_symptoms.sum(axis=1)

    if "swelling" in df.columns:
        df["swelling_ord"] = df["swelling"].map({"N": 0, "S": 1, "Y": 2})
    else:
        df["swelling_ord"] = 0
    df["symptom_severity"] = df["symptom_count"] + df["swelling_ord"]
    
    return df

# %% [Section 2-1] (추가) EDA 심화 분석 기반 피처 생성

def add_post_eda_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # ---------------------------------------------------------
    # 1. Treatment(치료)와 주요 수치 간의 역설적 상호작용 (PPT 12p)
    # ---------------------------------------------------------
    # 치료를 받았는데도 수치가 낮다면 더 위험할 수 있음을 반영
    # 수치형 변수 * (1 + Treatment) 형태로 가중치 부여
    
    # 치료 시 수치가 떨어지는 경향이 있던 변수들 (Protein, Blood cells)
    df['protein_treatment_interaction'] = df['protein_level'] * (1 + df['treatment'])
    df['blood_treatment_interaction']   = df['blood_cells'] * (1 + df['treatment'])
    
    # 치료 시 수치가 오르는 경향이 있던 변수들 (Chem_01, Trace_metal)
    df['chem01_treatment_interaction']  = df['chem_01'] * (1 + df['treatment'])
    df['trace_treatment_interaction']   = df['trace_metal'] * (1 + df['treatment'])

    # ---------------------------------------------------------
    # 2. 응고 지수 (Coagulation Index) - VIF 해결 및 주의 클래스 타겟팅 (PPT 9, 15p)
    # ---------------------------------------------------------
    # 두 변수의 비율이나 곱을 통해 새로운 지표 생성
    # (분모에 0이 들어가는 것을 방지하기 위해 1e-6 추가)
    df['coagulation_index'] = df['protein_level'] / (df['clot_time'] + 1e-6)
    df['coagulation_product'] = df['protein_level'] * df['clot_time']

    # ---------------------------------------------------------
    # 3. 치명적 증상 콤보 (Deadly Symptom Combo) (PPT 10p)
    # ---------------------------------------------------------
    # organ_enlarge(장기비대)와 swelling(부종)이 동시에 있을 때 가중치 폭발
    # swelling_ord: N=0, S=1, Y=2
    # organ_enlarge가 문자열(Y/N)일 수도 있고 숫자(1/0)일 수도 있음. 안전하게 변환.
    
    if df['organ_enlarge'].dtype == 'object':
        organ_val = (df['organ_enlarge'] == 'Y').astype(int)
    else:
        organ_val = df['organ_enlarge']
        
    # 부종 점수(0~2) * 장기비대(0~1) * 2 (가중치)
    # 둘 다 있으면 값이 커지고, 하나라도 없으면 0이 됨 (AND 조건 효과)
    df['deadly_combo'] = df['swelling_ord'] * organ_val * 2
    
    # ---------------------------------------------------------
    # 4. 관찰 일자(Obs_days)의 비선형성 (PPT 11p)
    # ---------------------------------------------------------
    # 관찰 기간이 길어질수록 위험도가 달라질 수 있음 (로그 변환으로 스케일 조정)
    df['log_obs_days'] = np.log1p(df['obs_days'])
    df['obs_days_squared'] = df['obs_days'] ** 2

    return df



# 3. 주의 구간 피처 (add_discrete_caution_features)
def add_discrete_caution_features(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    
    X["stage_mid"] = X["disease_stage"].between(2, 3).astype(int)
    X["beh_mid"]   = (X["behavior_index"] == 1).astype(int)

    X["caution_symptom_zone"] = (
        X["symptom_count"].between(1, 2) & (X["swelling_ord"] >= 1)
    ).astype(int)

    X["caution_stage_beh"] = (
        (X["stage_mid"] == 1) & (X["beh_mid"] == 1)
    ).astype(int)

    # organ_enlarge가 Y/N일 수도 있고 0/1일 수도 있음
    if X['organ_enlarge'].dtype == 'object':
        organ_flag = (X['organ_enlarge'] == 'Y')
    else:
        organ_flag = (X['organ_enlarge'] == 1)

    X["caution_organ_only"] = (
        organ_flag
        & (X["disease_stage"] == 1)
        & (X["behavior_index"] <= 1)
    ).astype(int)

    return X

# 4. [수정됨] 누수 없는(Leakage-Free) 그룹 통계
def add_group_stats_no_leakage(train: pd.DataFrame, test: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    train = train.copy()
    test = test.copy()
    
    bins = [60, 80, 90, 100, 130]
    labels = ["60-79", "80-89", "90-99", "100+"]
    
    for df in [train, test]:
        df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, right=False, include_lowest=True)

    group_cols = ['disease_stage', 'gender', 'age_group']
    agg_cols = ['enzyme_A', 'enzyme_B', 'protein_level', 'immune_index', 'lipid_index', 'blood_cells']

    for group_col in group_cols:
        for col in agg_cols:
            # Train 기준 통계 계산
            train_groupby = train.groupby(group_col)[col]
            mean_map = train_groupby.mean()
            std_map  = train_groupby.std()
            
            prefix = f'{col}_by_{group_col}'
            
            for df_target in [train, test]:
                # [핵심 수정] mapped_mean과 col_values 모두 float로 강제 변환 + values 연산 사용
                # map() 결과가 index 타입 등에 의해 category 연산으로 오해받는 것을 원천 차단
                
                mapped_mean = df_target[group_col].map(mean_map).astype(float).values
                mapped_std  = df_target[group_col].map(std_map).astype(float).values
                col_values  = df_target[col].astype(float).values
                
                # numpy array 끼리의 연산은 안전함
                df_target[f'{prefix}_diff_mean']  = col_values - mapped_mean
                df_target[f'{prefix}_zscore']     = (col_values - mapped_mean) / (mapped_std + 1e-6)
                df_target[f'{prefix}_ratio_mean'] = col_values / (mapped_mean + 1e-6)

    return train, test

# 5. 누수 없는 Q-Binning
def add_qbins_no_leakage(train: pd.DataFrame, test: pd.DataFrame, n_bins=4) -> (pd.DataFrame, pd.DataFrame):
    train = train.copy()
    test = test.copy()
    
    lab_cols = [
        "chem_01", "chem_02", "protein_level", "trace_metal",
        "enzyme_A", "enzyme_B", "lipid_index", "blood_cells", "clot_time"
    ]
    
    for col in lab_cols:
        if col not in train.columns: continue
        
        # Train 기준 구간(bins) 계산
        try:
            _, bins = pd.qcut(train[col], q=n_bins, retbins=True, duplicates='drop')
        except ValueError:
            continue
            
        # Train/Test에 동일한 bins 적용
        # labels=False로 설정하여 int형으로 반환
        train[f"{col}_qbin"] = pd.cut(train[col], bins=bins, labels=False, include_lowest=True).fillna(-1).astype(int)
        test[f"{col}_qbin"] = pd.cut(test[col], bins=bins, labels=False, include_lowest=True).fillna(-1).astype(int)
        
    return train, test

In [5]:
# %% [Section 3] 전처리 실행

# 1. 기본 전처리 (나이 변환)
def prepare_raw_simple(df):
    df = df.copy()
    if "birth_date" in df.columns:
        df["birth_date"] = pd.to_datetime(df["birth_date"])
        df["age"] = 2025 - df["birth_date"].dt.year
    
    # gender 숫자 변환 (XGB용, CatBoost용은 원본 gender를 쓰지만 여기선 미리 만들어둠)
    if "gender" in df.columns:
        df["gender_int"] = df["gender"].map({"M": 0, "F": 1})
    return df

print("1. 기본 전처리 중...")
train_df = prepare_raw_simple(train_df)
test_df = prepare_raw_simple(test_df)

print("2. EDA Rule 적용 중...")
train_df = add_eda_rule_features_v2(train_df)
test_df  = add_eda_rule_features_v2(test_df)

print("3. Boosting Feature 생성 중...")
train_df = make_boosting_features_v2(train_df)
test_df  = make_boosting_features_v2(test_df)

print("4. Caution Feature 생성 중...")
train_df = add_discrete_caution_features(train_df)
test_df  = add_discrete_caution_features(test_df)

print("4-1. EDA 심화 피처(Treatment Interaction 등) 생성 중...")
train_df = add_post_eda_features(train_df)
test_df  = add_post_eda_features(test_df)

print("5. Group Stats (No Leakage) 생성 중...")
# 여기서 에러가 났었으므로, 위에서 수정한 함수가 호출됩니다.
train_df, test_df = add_group_stats_no_leakage(train_df, test_df)

print("6. Q-Binning (No Leakage) 생성 중...")
train_df, test_df = add_qbins_no_leakage(train_df, test_df)

# 7. X, y 분리
drop_cols = ["index", "name", "birth_date", "geo_code", "birth_date"]
y = train_df["target"].values
X_all = train_df.drop(columns=drop_cols + ["target"], errors='ignore')
X_all_test = test_df.drop(columns=drop_cols, errors='ignore')

print(f"Preprocessed X_all shape: {X_all.shape}")
print(f"Preprocessed X_all_test shape: {X_all_test.shape}")

1. 기본 전처리 중...
2. EDA Rule 적용 중...
3. Boosting Feature 생성 중...
4. Caution Feature 생성 중...
4-1. EDA 심화 피처(Treatment Interaction 등) 생성 중...
5. Group Stats (No Leakage) 생성 중...
6. Q-Binning (No Leakage) 생성 중...
Preprocessed X_all shape: (6500, 123)
Preprocessed X_all_test shape: (1405, 123)


In [6]:
# %% [Section 4] 스마트 스케일링 (Skewness 기반)

# 1. 스케일링 대상: 순수 수치형 데이터만 (범주형, 바이너리 제외)
numeric_candidates = X_all.select_dtypes(include=['number']).columns

EXCLUDE_COLS = [
    "gender_int", "fluid_accum", "organ_enlarge", "vascular_marks", 
    "swelling", "treatment", "geo_code", "risk_segment",
    "chem_01_range", "trace_metal_range", "swelling_ord",
    "target", "index"
]

NUMERIC_COLS = [c for c in numeric_candidates if c not in EXCLUDE_COLS and not c.endswith('_qbin')]

# 2. 스케일링 함수
def apply_skew_scaling(train_df, test_df, cols):
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    # Train 기준 Skewness 계산
    skewness = train_df[cols].skew()
    print(f"Applying Skew-based Scaling on {len(cols)} columns...")
    
    for col in cols:
        skew = abs(skewness[col])
        
        if skew < 0.5:
            scaler = StandardScaler()
        elif skew < 1.5:
            scaler = RobustScaler()
        else:
            scaler = QuantileTransformer(output_distribution="normal", random_state=42)
            
        train_df[col] = scaler.fit_transform(train_df[[col]]).flatten()
        test_df[col] = scaler.transform(test_df[[col]]).flatten()
            
    return train_df, test_df

# 3. 적용
X_all, X_all_test = apply_skew_scaling(X_all, X_all_test, NUMERIC_COLS)
print("Smart Scaling Complete.")

Applying Skew-based Scaling on 99 columns...
Smart Scaling Complete.


In [36]:
# %% [Section 5-1 (최종)] 전체 피처 대상 Forward Selection (상세 로그)

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. 전체 피처 중 학습 가능한 수치형만 골라내기
numeric_cols_for_selection = X_all.select_dtypes(include=['number']).columns.tolist()
exclude_cols = ['target', 'index']
numeric_cols_for_selection = [c for c in numeric_cols_for_selection if c not in exclude_cols]

# 전체 수치형 데이터로 X_selection 구성
X_selection = X_all[numeric_cols_for_selection].copy()
y_selection = y

print(f"🚀 전체 수치형 피처 풀(Pool) 크기: {X_selection.shape[1]}개")

# 2. 기초 중요도 산출 (한 번 전체 학습)
print("\n[1단계] 기초 중요도 산출을 위한 전체 학습 진행 중...")
temp_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    objective='multi:softprob',
    eval_metric='mlogloss',
    tree_method='hist',
    random_state=42,
    n_jobs=-1
)
temp_model.fit(X_selection, y_selection)

# 중요도 추출 및 정렬
importances = pd.DataFrame({
    'feature': X_selection.columns,
    'importance': temp_model.feature_importances_
}).sort_values(by='importance', ascending=False)

# 중요도가 0인(아예 안 쓰인) 피처는 제거
importances = importances[importances['importance'] > 0]
sorted_features = importances['feature'].tolist()
total_candidates = len(sorted_features)

print(f"--> 중요도가 0보다 큰 유의미한 피처: {total_candidates}개")
print(f"--> Top 5: {sorted_features[:5]}")

# 3. Forward Selection (하나씩 추가하며 성능 체크)
print(f"\n[2단계] 최적 조합 찾기 (Forward Selection) 시작...")

best_score = 0.0
best_features = []
history_scores = []
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

check_limit = len(sorted_features)

# 연속으로 성능 향상이 없는 횟수 카운트 (Early Stopping용)
no_improvement_count = 0
PATIENCE = 20  # 5번 연속으로 성능 안 오르면 "이제 그만하자" 하고 종료

for i, feature in enumerate(sorted_features[:check_limit]):
    current_features = best_features + [feature]
    
    # 데이터프레임 슬라이싱 (.values 사용으로 속도 최적화)
    X_current = X_selection[current_features].values
    
    fold_scores = []
    for train_idx, val_idx in kf.split(X_current, y_selection):
        model = XGBClassifier(
            n_estimators=200, # 속도 위해 가볍게
            learning_rate=0.05,
            objective='multi:softprob',
            tree_method='hist',
            random_state=42,
            n_jobs=-1,
            verbosity=0
        )
        model.fit(X_current[train_idx], y_selection[train_idx])
        preds = model.predict_proba(X_current[val_idx])
        
        if preds.shape[1] > 2:
            y_bin = label_binarize(y_selection[val_idx], classes=np.unique(y_selection))
            score = average_precision_score(y_bin, preds, average="macro")
        else:
            score = average_precision_score(y_selection[val_idx], preds[:, 1], average="macro")
        fold_scores.append(score)
    
    mean_score = np.mean(fold_scores)
    history_scores.append(mean_score)
    
    # 출력 메시지 구성
    status_msg = f"[{i+1}/{check_limit}] 검사중: {feature:<20}"
    
    if mean_score > best_score:
        diff = mean_score - best_score
        best_score = mean_score
        best_features.append(feature)
        no_improvement_count = 0 # 성능 올랐으니 카운트 초기화
        print(f"{status_msg} | Score: {mean_score:.5f} (▲ +{diff:.5f}) [선택됨! 현재 {len(best_features)}개]")
    else:
        no_improvement_count += 1
        # 성능이 안 오르면 추가하지 않음 (Pass)
        # print(f"{status_msg} | Score: {mean_score:.5f} (  Pass ) [연속실패: {no_improvement_count}]")
        pass
    
    # (옵션) 만약 성능이 5번 연속으로 안 오르면 조기 종료하고 싶다면?
    #if no_improvement_count >= PATIENCE:
     #    print(f"\n⏹️ {PATIENCE}회 연속 성능 향상 없음. 조기 종료합니다.")
      #   break

print("\n" + "="*40)
print(f"🎉 최종 결과: {total_candidates}개 후보 중 {len(best_features)}개 선택됨")
print(f"🏆 최고 CV 점수: {best_score:.5f}")
print("="*40)
print("Selected Features:", best_features)

# 4. X_xgb를 이 '최강 조합'으로 업데이트!
X_xgb = X_selection[best_features].copy()
X_xgb_test = X_all_test[best_features].copy()
print(f"✅ X_xgb가 최적 피처 {len(best_features)}개로 업데이트되었습니다!")

🚀 전체 수치형 피처 풀(Pool) 크기: 114개

[1단계] 기초 중요도 산출을 위한 전체 학습 진행 중...
--> 중요도가 0보다 큰 유의미한 피처: 105개
--> Top 5: ['chem_01_range', 'chem01_trace_combo', 'chem_01', 'symptom_count', 'clot_time_qbin']

[2단계] 최적 조합 찾기 (Forward Selection) 시작...
[1/105] 검사중: chem_01_range        | Score: 0.47901 (▲ +0.47901) [선택됨! 현재 1개]
[2/105] 검사중: chem01_trace_combo   | Score: 0.55503 (▲ +0.07603) [선택됨! 현재 2개]
[3/105] 검사중: chem_01              | Score: 0.57819 (▲ +0.02316) [선택됨! 현재 3개]
[4/105] 검사중: symptom_count        | Score: 0.59617 (▲ +0.01798) [선택됨! 현재 4개]
[5/105] 검사중: clot_time_qbin       | Score: 0.61275 (▲ +0.01658) [선택됨! 현재 5개]
[6/105] 검사중: symptom_severity     | Score: 0.61415 (▲ +0.00140) [선택됨! 현재 6개]
[7/105] 검사중: health_per_stage     | Score: 0.61819 (▲ +0.00404) [선택됨! 현재 7개]
[9/105] 검사중: obs_days             | Score: 0.64047 (▲ +0.02229) [선택됨! 현재 8개]
[10/105] 검사중: swelling_ord         | Score: 0.64277 (▲ +0.00230) [선택됨! 현재 9개]
[12/105] 검사중: gender_int           | Score: 0.64529 (▲ +0.00252) [선택됨! 현재 

In [5]:
# %% [Section 5-2] LightGBM 전체 피처 대상 Forward Selection

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. 전체 피처 중 학습 가능한 수치형만 골라내기
# (XGBoost와 동일한 풀에서 시작)
numeric_cols_for_selection = X_all.select_dtypes(include=['number']).columns.tolist()
exclude_cols = ['target', 'index']
numeric_cols_for_selection = [c for c in numeric_cols_for_selection if c not in exclude_cols]

# 전체 수치형 데이터로 X_selection 구성
X_selection = X_all[numeric_cols_for_selection].copy()
y_selection = y

print(f"🚀 [LGBM] 전체 수치형 피처 풀(Pool) 크기: {X_selection.shape[1]}개")

# 2. 기초 중요도 산출 (한 번 전체 학습)
print("\n[1단계] 기초 중요도 산출을 위한 전체 학습 진행 중...")
temp_model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    objective='multiclass',
    random_state=42,
    n_jobs=-1,
    verbosity=-1
)
temp_model.fit(X_selection, y_selection)

# 중요도 추출 및 정렬
importances = pd.DataFrame({
    'feature': X_selection.columns,
    'importance': temp_model.feature_importances_
}).sort_values(by='importance', ascending=False)

# 중요도가 0인(아예 안 쓰인) 피처는 제거
importances = importances[importances['importance'] > 0]
sorted_features = importances['feature'].tolist()
total_candidates = len(sorted_features)

print(f"--> 중요도가 0보다 큰 유의미한 피처: {total_candidates}개")
print(f"--> Top 5: {sorted_features[:5]}")

# 3. Forward Selection (하나씩 추가하며 성능 체크)
print(f"\n[2단계] 최적 조합 찾기 (Forward Selection) 시작...")
print(f"    검사 대상: 상위 {min(60, total_candidates)}개 (전체 {total_candidates}개 중)")

best_score = 0.0
best_features = []
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 검사할 최대 개수 설정 (시간 절약을 위해 상위 60개만 볼지, 전체 다 볼지 결정)
check_limit = len(sorted_features) 

# 연속으로 성능 향상이 없는 횟수 카운트 (Early Stopping용)
no_improvement_count = 0
PATIENCE = 20  # 20번 연속으로 성능 안 오르면 종료

for i, feature in enumerate(sorted_features[:check_limit]):
    current_features = best_features + [feature]
    
    # 데이터프레임 슬라이싱 (.values 사용으로 속도 최적화)
    X_current = X_selection[current_features].values
    
    fold_scores = []
    for train_idx, val_idx in kf.split(X_current, y_selection):
        model = LGBMClassifier(
            n_estimators=200, # 속도 위해 가볍게
            learning_rate=0.05,
            objective='multiclass',
            random_state=42,
            n_jobs=-1,
            verbosity=-1
        )
        model.fit(X_current[train_idx], y_selection[train_idx])
        preds = model.predict_proba(X_current[val_idx])
        
        if preds.shape[1] > 2:
            y_bin = label_binarize(y_selection[val_idx], classes=np.unique(y_selection))
            score = average_precision_score(y_bin, preds, average="macro")
        else:
            score = average_precision_score(y_selection[val_idx], preds[:, 1], average="macro")
        fold_scores.append(score)
    
    mean_score = np.mean(fold_scores)
    
    # 출력 메시지 구성
    status_msg = f"[{i+1}/{check_limit}] 검사중: {feature:<20}"
    
    if mean_score > best_score:
        diff = mean_score - best_score
        best_score = mean_score
        best_features.append(feature)
        no_improvement_count = 0 # 성능 올랐으니 카운트 초기화
        print(f"{status_msg} | Score: {mean_score:.5f} (▲ +{diff:.5f}) [선택됨! 현재 {len(best_features)}개]")
    else:
        no_improvement_count += 1
        # 성능이 안 오르면 추가하지 않음 (Pass)
        # print(f"{status_msg} | Score: {mean_score:.5f} (  Pass ) [연속실패: {no_improvement_count}]")
        pass
    
    # 조기 종료 조건
    #if no_improvement_count >= PATIENCE:
    #    print(f"\n⏹️ {PATIENCE}회 연속 성능 향상 없음. 조기 종료합니다.")
    #    break

print("\n" + "="*40)
print(f"🎉 [LGBM] 최종 결과: {len(best_features)}개 피처 선택됨")
print(f"🏆 최고 CV 점수: {best_score:.5f}")
print("="*40)
print("Selected Features:", best_features)

# 4. X_lgb를 이 '최강 조합'으로 업데이트!
X_lgb = X_selection[best_features].copy()
X_lgb_test = X_all_test[best_features].copy()
print(f"✅ X_lgb가 최적 피처 {len(best_features)}개로 업데이트되었습니다!")

🚀 [LGBM] 전체 수치형 피처 풀(Pool) 크기: 114개

[1단계] 기초 중요도 산출을 위한 전체 학습 진행 중...
--> 중요도가 0보다 큰 유의미한 피처: 112개
--> Top 5: ['obs_days', 'clot_time', 'age', 'chem02_trace_ratio', 'obs_per_age']

[2단계] 최적 조합 찾기 (Forward Selection) 시작...
    검사 대상: 상위 60개 (전체 112개 중)
[1/112] 검사중: obs_days             | Score: 0.55987 (▲ +0.55987) [선택됨! 현재 1개]
[2/112] 검사중: clot_time            | Score: 0.57709 (▲ +0.01722) [선택됨! 현재 2개]
[3/112] 검사중: age                  | Score: 0.59666 (▲ +0.01958) [선택됨! 현재 3개]
[4/112] 검사중: chem02_trace_ratio   | Score: 0.61422 (▲ +0.01756) [선택됨! 현재 4개]
[8/112] 검사중: lipid_blood_ratio    | Score: 0.62240 (▲ +0.00818) [선택됨! 현재 5개]
[9/112] 검사중: disease_velocity     | Score: 0.62651 (▲ +0.00411) [선택됨! 현재 6개]
[10/112] 검사중: trace_metal          | Score: 0.64809 (▲ +0.02159) [선택됨! 현재 7개]
[11/112] 검사중: chem01_treatment_interaction | Score: 0.65770 (▲ +0.00961) [선택됨! 현재 8개]
[15/112] 검사중: blood_treatment_interaction | Score: 0.66644 (▲ +0.00874) [선택됨! 현재 9개]
[18/112] 검사중: health_per_stage     |

In [6]:
# %% [Section 5-3 (무제한)] CatBoost 전체 피처 대상 Forward Selection (제한 없음)

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. 전체 피처 풀 구성
numeric_candidates = X_all.select_dtypes(include=['number']).columns.tolist()
exclude_nums = ['target', 'index', 'gender_int'] 
numeric_cols = [c for c in numeric_candidates if c not in exclude_nums]

CAT_FEATURES = [
    'gender', 'fluid_accum', 'swelling', 
    'risk_segment_str', 'chem_01_range_str', 'trace_metal_range_str'
]
CAT_FEATURES = [c for c in CAT_FEATURES if c in X_all.columns]

X_selection = X_all[numeric_cols + CAT_FEATURES].copy()
y_selection = y

print(f"🚀 [CatBoost] 전체 피처 풀 크기: {X_selection.shape[1]}개")

# 2. 기초 중요도 산출
print("\n[1단계] 기초 중요도 산출을 위한 전체 학습 진행 중...")

train_pool = Pool(X_selection, y_selection, cat_features=CAT_FEATURES)

temp_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_state=42,
    thread_count=-1,
    verbose=False
)
temp_model.fit(train_pool)

importances = pd.DataFrame({
    'feature': X_selection.columns,
    'importance': temp_model.get_feature_importance()
}).sort_values(by='importance', ascending=False)

importances = importances[importances['importance'] > 0]
sorted_features = importances['feature'].tolist()
total_candidates = len(sorted_features)

print(f"--> 중요도가 0보다 큰 유의미한 피처: {total_candidates}개")
print(f"--> Top 5: {sorted_features[:5]}")

# 3. Forward Selection (제한 없이 끝까지!)
print(f"\n[2단계] 최적 조합 찾기 (Forward Selection) 시작...")
print(f"    검사 대상: 전체 {total_candidates}개 (끝까지 갑니다 🔥)")

best_score = 0.0
best_features = []
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# [수정 포인트 1] 60개 제한 해제 -> 전체 다 검사
check_limit = len(sorted_features) 

# [수정 포인트 2] 조기 종료 카운터 삭제 (끝까지 돌릴 거니까 필요 없음)

for i, feature in enumerate(sorted_features[:check_limit]):
    current_features = best_features + [feature]
    
    current_cat_features = [f for f in current_features if f in CAT_FEATURES]
    X_current = X_selection[current_features]
    
    fold_scores = []
    for train_idx, val_idx in kf.split(X_current, y_selection):
        X_tr, X_val = X_current.iloc[train_idx], X_current.iloc[val_idx]
        y_tr, y_val = y_selection[train_idx], y_selection[val_idx]
        
        train_pool_fold = Pool(X_tr, y_tr, cat_features=current_cat_features)
        val_pool_fold   = Pool(X_val, y_val, cat_features=current_cat_features)
        
        model = CatBoostClassifier(
            iterations=200, 
            learning_rate=0.05,
            loss_function='MultiClass',
            eval_metric='MultiClass',
            random_state=42,
            thread_count=-1,
            verbose=False,
            allow_writing_files=False
        )
        
        model.fit(train_pool_fold)
        preds = model.predict_proba(val_pool_fold)
        
        if preds.shape[1] > 2:
            y_bin = label_binarize(y_val, classes=np.unique(y_selection))
            score = average_precision_score(y_bin, preds, average="macro")
        else:
            score = average_precision_score(y_val, preds[:, 1], average="macro")
        fold_scores.append(score)
    
    mean_score = np.mean(fold_scores)
    status_msg = f"[{i+1}/{check_limit}] 검사중: {feature:<20}"
    
    if mean_score > best_score:
        diff = mean_score - best_score
        best_score = mean_score
        best_features.append(feature)
        print(f"{status_msg} | Score: {mean_score:.5f} (▲ +{diff:.5f}) [선택됨! 현재 {len(best_features)}개]")
    else:
        # 성능 안 오르면 그냥 넘어감 (조기 종료 없음)
        # print(f"{status_msg} | Score: {mean_score:.5f} (  Pass )")
        pass
    
    # [수정 포인트 3] 조기 종료 코드(if no_improvement >= PATIENCE) 삭제됨

print("\n" + "="*40)
print(f"🎉 [CatBoost] 최종 결과: {len(best_features)}개 피처 선택됨")
print(f"🏆 최고 CV 점수: {best_score:.5f}")
print("="*40)
print("Selected Features:", best_features)

# 4. X_cat 업데이트
X_cat = X_selection[best_features].copy()
X_cat_test = X_cat_test[best_features].copy()
CAT_FEATS_FINAL = [f for f in best_features if f in CAT_FEATURES]

print(f"✅ X_cat 업데이트 완료! (최종 범주형 {len(CAT_FEATS_FINAL)}개 포함)")

🚀 [CatBoost] 전체 피처 풀 크기: 119개

[1단계] 기초 중요도 산출을 위한 전체 학습 진행 중...
--> 중요도가 0보다 큰 유의미한 피처: 118개
--> Top 5: ['age', 'chem_01', 'clot_time', 'health_per_stage', 'chem01_enzymeB_combo']

[2단계] 최적 조합 찾기 (Forward Selection) 시작...
    검사 대상: 전체 118개 (끝까지 갑니다 🔥)
[1/118] 검사중: age                  | Score: 0.43702 (▲ +0.43702) [선택됨! 현재 1개]
[2/118] 검사중: chem_01              | Score: 0.59095 (▲ +0.15393) [선택됨! 현재 2개]
[3/118] 검사중: clot_time            | Score: 0.59984 (▲ +0.00888) [선택됨! 현재 3개]
[4/118] 검사중: health_per_stage     | Score: 0.61594 (▲ +0.01610) [선택됨! 현재 4개]
[5/118] 검사중: chem01_enzymeB_combo | Score: 0.62290 (▲ +0.00696) [선택됨! 현재 5개]
[6/118] 검사중: chem01_treatment_interaction | Score: 0.62396 (▲ +0.00106) [선택됨! 현재 6개]
[7/118] 검사중: obs_per_age          | Score: 0.63468 (▲ +0.01072) [선택됨! 현재 7개]
[10/118] 검사중: chem01_trace_combo   | Score: 0.63718 (▲ +0.00250) [선택됨! 현재 8개]
[13/118] 검사중: disease_velocity     | Score: 0.64234 (▲ +0.00516) [선택됨! 현재 9개]
[16/118] 검사중: chem02_trace_ratio   | Score:

NameError: name 'X_cat_test' is not defined

##### (cat 안씀)

In [10]:
# %% [최종] XGBoost Optuna 튜닝 (200회)

import optuna
import numpy as np
import pandas as pd
from optuna.samplers import TPESampler
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize

# 1. 피처 확정 (사용자님 리스트)
XGB_FEATS_SELECTED = [
    'chem_01_range', 'chem01_trace_combo', 'chem_01', 'symptom_count', 'clot_time_qbin', 
    'symptom_severity', 'health_per_stage', 'obs_days', 'swelling_ord', 'gender_int', 
    'age', 'lipid_index_by_gender_ratio_mean', 'chem_01_qbin', 'enzyme_B_by_disease_stage_zscore', 
    'blood_cells_by_gender_zscore', 'enzyme_A_by_gender_diff_mean', 'chem01_treatment_interaction', 
    'deadly_combo', 'enzyme_A_by_disease_stage_ratio_mean', 'enzyme_B', 'sum_enzyme', 
    'chem01_chem02_combo', 'sum_chem', 'blood_cells_by_gender_ratio_mean', 
    'trace_treatment_interaction', 'enzyme_B_by_age_group_ratio_mean', 
    'lipid_index_by_disease_stage_ratio_mean', 'blood_cells_by_age_group_diff_mean', 
    'enzyme_A', 'enzyme_B_by_disease_stage_diff_mean', 'blood_treatment_interaction', 
    'blood_cells_by_disease_stage_diff_mean', 'enzyme_ratio', 'behavior_per_age', 
    'lipid_index_qbin', 'treatment'
]

# 데이터셋 업데이트 (혹시 모를 에러 방지용 필터링 포함)
valid_feats = [c for c in XGB_FEATS_SELECTED if c in X_all.columns]
X_xgb = X_all[valid_feats].copy()
X_xgb_test = X_all_test[valid_feats].copy()

print(f"✅ X_xgb 데이터셋 준비 완료! (피처 {X_xgb.shape[1]}개)")

# 2. mAP 평가 함수
def score_function(y_true, y_pred_proba):
    if y_pred_proba.shape[1] > 2:
        y_bin = label_binarize(y_true, classes=np.unique(y_true))
        return average_precision_score(y_bin, y_pred_proba, average="macro")
    else:
        return average_precision_score(y_true, y_pred_proba[:, 1], average="macro")

# 3. Optuna 목적 함수
def objective_xgb(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'gamma': trial.suggest_float('gamma', 0.0, 3.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        # 고정 파라미터
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'tree_method': 'hist',
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    X_vals = X_xgb.values
    y_vals = y

    for train_idx, val_idx in skf.split(X_vals, y_vals):
        model = XGBClassifier(**param)
        model.fit(X_vals[train_idx], y_vals[train_idx])
        preds = model.predict_proba(X_vals[val_idx])
        scores.append(score_function(y_vals[val_idx], preds))

    return np.mean(scores)

# 4. 실행 (200회)
print("🚀 XGBoost Optuna 튜닝 시작 (총 200회)...")
study_xgb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_xgb.optimize(objective_xgb, n_trials=200, show_progress_bar=True)

print(f"\n🏆 Best mAP: {study_xgb.best_value:.6f}")
print("Best Params:")
print(study_xgb.best_params)

# 5. 최적 파라미터 저장
best_params_xgb = study_xgb.best_params
best_params_xgb.update({
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'tree_method': 'hist',
    'random_state': 42,
    'n_jobs': -1
})

[I 2025-11-25 05:12:17,043] A new study created in memory with name: no-name-48b44d1d-fa38-44a8-93d9-9b95ff593512


✅ X_xgb 데이터셋 준비 완료! (피처 36개)
🚀 XGBoost Optuna 튜닝 시작 (총 200회)...


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2025-11-25 05:12:30,839] Trial 0 finished with value: 0.6835296504474601 and parameters: {'n_estimators': 1062, 'learning_rate': 0.0862735828664018, 'max_depth': 10, 'min_child_weight': 9, 'gamma': 0.46805592132730955, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'reg_alpha': 2.9154431891537547, 'reg_lambda': 0.2537815508265665}. Best is trial 0 with value: 0.6835296504474601.
[I 2025-11-25 05:13:27,009] Trial 1 finished with value: 0.6878880926174746 and parameters: {'n_estimators': 1562, 'learning_rate': 0.005318033256270142, 'max_depth': 12, 'min_child_weight': 13, 'gamma': 0.6370173320348285, 'subsample': 0.5909124836035503, 'colsample_bytree': 0.5917022549267169, 'reg_alpha': 0.016480446427978974, 'reg_lambda': 0.12561043700013558}. Best is trial 1 with value: 0.6878880926174746.
[I 2025-11-25 05:13:56,705] Trial 2 finished with value: 0.6936684413844828 and parameters: {'n_estimators': 1148, 'learning_rate': 0.011963764382790322, 'max_depth': 9, 'mi

In [11]:
# %% [최종] LightGBM Optuna 튜닝 (200회)

import optuna
import numpy as np
import pandas as pd
from optuna.samplers import TPESampler
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize

# 1. 피처 확정 (사용자님 리스트)
LGBM_FEATS_SELECTED = [
    'obs_days', 'clot_time', 'age', 'chem02_trace_ratio', 'lipid_blood_ratio', 
    'disease_velocity', 'trace_metal', 'chem01_treatment_interaction', 
    'blood_treatment_interaction', 'health_per_stage', 'trace_treatment_interaction', 
    'chem_02', 'diff_chem', 'diff_enzyme', 'enzyme_B_by_age_group_diff_mean', 
    'sum_enzyme', 'enzyme_A_by_gender_zscore', 'log_obs_days', 'obs_days_squared'
]

# 데이터셋 업데이트 (혹시 모를 에러 방지용 필터링 포함)
# X_all은 앞서 전처리 단계에서 이미 만들어져 있어야 합니다.
valid_feats = [c for c in LGBM_FEATS_SELECTED if c in X_all.columns]

# 누락된 피처 확인
if len(valid_feats) < len(LGBM_FEATS_SELECTED):
    print("⚠️ 경고: 일부 피처가 X_all에 없습니다:", set(LGBM_FEATS_SELECTED) - set(valid_feats))

X_lgb = X_all[valid_feats].copy()
X_lgb_test = X_all_test[valid_feats].copy()

print(f"✅ X_lgb 데이터셋 준비 완료! (피처 {X_lgb.shape[1]}개)")

# 2. mAP 평가 함수
def score_function(y_true, y_pred_proba):
    if y_pred_proba.shape[1] > 2:
        y_bin = label_binarize(y_true, classes=np.unique(y_true))
        return average_precision_score(y_bin, y_pred_proba, average="macro")
    else:
        return average_precision_score(y_true, y_pred_proba[:, 1], average="macro")

# 3. Optuna 목적 함수
def objective_lgb(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        # 고정 파라미터
        'objective': 'multiclass',
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': -1
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    X_vals = X_lgb.values
    y_vals = y

    for train_idx, val_idx in skf.split(X_vals, y_vals):
        model = LGBMClassifier(**param)
        model.fit(X_vals[train_idx], y_vals[train_idx])
        preds = model.predict_proba(X_vals[val_idx])
        scores.append(score_function(y_vals[val_idx], preds))

    return np.mean(scores)

# 4. 실행 (200회)
print("🚀 LightGBM Optuna 튜닝 시작 (총 200회)...")
study_lgb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_lgb.optimize(objective_lgb, n_trials=200, show_progress_bar=True)

print(f"\n🏆 Best mAP: {study_lgb.best_value:.6f}")
print("Best Params:")
print(study_lgb.best_params)

# 5. 최적 파라미터 저장
best_params_lgb = study_ vm,lgb.best_params
best_params_lgb.update({
    'objective': 'multiclass',
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
})

[I 2025-11-25 07:12:01,506] A new study created in memory with name: no-name-91de0c3a-3cb0-45e6-89e1-fc4fa3863ec1


✅ X_lgb 데이터셋 준비 완료! (피처 19개)
🚀 LightGBM Optuna 튜닝 시작 (총 200회)...


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2025-11-25 07:12:05,800] Trial 0 finished with value: 0.6843232820997651 and parameters: {'n_estimators': 1062, 'learning_rate': 0.0862735828664018, 'num_leaves': 152, 'max_depth': 10, 'min_child_samples': 24, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'reg_alpha': 2.9154431891537547, 'reg_lambda': 0.2537815508265665}. Best is trial 0 with value: 0.6843232820997651.
[I 2025-11-25 07:12:47,182] Trial 1 finished with value: 0.6985382295983816 and parameters: {'n_estimators': 1562, 'learning_rate': 0.005318033256270142, 'num_leaves': 195, 'max_depth': 13, 'min_child_samples': 29, 'subsample': 0.5909124836035503, 'colsample_bytree': 0.5917022549267169, 'reg_alpha': 0.016480446427978974, 'reg_lambda': 0.12561043700013558}. Best is trial 1 with value: 0.6985382295983816.
[I 2025-11-25 07:12:55,515] Trial 2 finished with value: 0.6898610891325383 and parameters: {'n_estimators': 1148, 'learning_rate': 0.011963764382790322, 'num_leaves': 130, 'max_depth': 4, 'm

---------------------------------------------------------------------


In [8]:
XGB_FEATS_SELECTED = [
    'chem_01_range', 'chem01_trace_combo', 'chem_01', 'symptom_count', 'clot_time_qbin', 
    'symptom_severity', 'health_per_stage', 'obs_days', 'swelling_ord', 'gender_int', 
    'age', 'lipid_index_by_gender_ratio_mean', 'chem_01_qbin', 'enzyme_B_by_disease_stage_zscore', 
    'blood_cells_by_gender_zscore', 'enzyme_A_by_gender_diff_mean', 'chem01_treatment_interaction', 
    'deadly_combo', 'enzyme_A_by_disease_stage_ratio_mean', 'enzyme_B', 'sum_enzyme', 
    'chem01_chem02_combo', 'sum_chem', 'blood_cells_by_gender_ratio_mean', 
    'trace_treatment_interaction', 'enzyme_B_by_age_group_ratio_mean', 
    'lipid_index_by_disease_stage_ratio_mean', 'blood_cells_by_age_group_diff_mean', 
    'enzyme_A', 'enzyme_B_by_disease_stage_diff_mean', 'blood_treatment_interaction', 
    'blood_cells_by_disease_stage_diff_mean', 'enzyme_ratio', 'behavior_per_age', 
    'lipid_index_qbin', 'treatment'
]

LGBM_FEATS_SELECTED = [
    'obs_days', 'clot_time', 'age', 'chem02_trace_ratio', 'lipid_blood_ratio', 
    'disease_velocity', 'trace_metal', 'chem01_treatment_interaction', 
    'blood_treatment_interaction', 'health_per_stage', 'trace_treatment_interaction', 
    'chem_02', 'diff_chem', 'diff_enzyme', 'enzyme_B_by_age_group_diff_mean', 
    'sum_enzyme', 'enzyme_A_by_gender_zscore', 'log_obs_days', 'obs_days_squared'
]

In [9]:
final_params_xgb = {
    'n_estimators': 788,
    'learning_rate': 0.06238066751566709,
    'max_depth': 6,
    'min_child_weight': 1,
    'gamma': 0.3960858015669083,
    'subsample': 0.9214205395769955,
    'colsample_bytree': 0.5491320112459366,
    'reg_alpha': 0.002669920381142417,
    'reg_lambda': 0.015444177908340267,
    # 고정값
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'tree_method': 'hist',
    'random_state': 42,
    'n_jobs': -1
}

final_params_lgb = {
    'n_estimators': 912,
    'learning_rate': 0.01090933368427726,
    'num_leaves': 193,
    'max_depth': 14,
    'min_child_samples': 51,
    'subsample': 0.5933220031847126,
    'colsample_bytree': 0.5036928973563994,
    'reg_alpha': 0.008835424632453443,
    'reg_lambda': 0.07034822683895961,
    # 고정값
    'objective': 'multiclass',
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
}

In [10]:
# %% [Step 1] OOF(Out-of-Fold) 생성

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
import numpy as np

def get_oof_predictions(model, X, y, X_test, n_splits=5, random_state=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # OOF 저장할 배열 (Train 크기)
    oof_train = np.zeros((X.shape[0], 3)) # Class가 3개이므로 (N, 3)
    # Test 예측값 누적할 배열
    oof_test_skf = np.zeros((X_test.shape[0], 3))
    
    print(f"Creating OOF for {model.__class__.__name__}...", end="")
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        # 데이터 분할
        X_tr, y_tr = X.iloc[train_idx], y[train_idx]
        X_val = X.iloc[val_idx]
        
        # 모델 복제 및 학습
        clf = clone(model)
        clf.fit(X_tr, y_tr)
        
        # Validation 예측 (OOF)
        oof_train[val_idx] = clf.predict_proba(X_val)
        
        # Test 예측 (누적해서 나중에 평균냄)
        oof_test_skf += clf.predict_proba(X_test) / n_splits
        print(f".", end="")
        
    print(" Done!")
    return oof_train, oof_test_skf

# 1. 모델 정의 (이미 튜닝된 final_params 사용)
xgb_model = XGBClassifier(**final_params_xgb)
lgb_model = LGBMClassifier(**final_params_lgb)

# 2. OOF 생성 실행
# (X_xgb, X_lgb는 각각 최적 피처로 준비된 상태여야 함)
print("🚀 OOF 생성 시작...")
oof_train_xgb, oof_test_xgb = get_oof_predictions(xgb_model, X_xgb, y, X_xgb_test)
oof_train_lgb, oof_test_lgb = get_oof_predictions(lgb_model, X_lgb, y, X_lgb_test)

print("\n✅ OOF 생성 완료!")

🚀 OOF 생성 시작...


NameError: name 'X_xgb' is not defined

In [1]:
# %% [Step 2-1] 최적 가중치 찾기 (Weighted Blending)

best_score = 0
best_weight = 0

# 0.0 ~ 1.0 사이를 0.01 단위로 훑으면서 최고의 조합 찾기
for w in np.arange(0.0, 1.01, 0.001):
    # 가중치 적용 (XGB * w + LGB * (1-w))
    blended_pred = (oof_train_xgb * w) + (oof_train_lgb * (1 - w))
    
    # 점수 계산
    current_score = score_function(y, blended_pred)
    
    if current_score > best_score:
        best_score = current_score
        best_weight = w

print(f"🏆 최적 가중치 발견! XGB: {best_weight:.2f}, LGB: {1-best_weight:.2f}")
print(f"   그때의 CV mAP: {best_score:.6f}")

# 최종 Test 예측 (찾은 가중치 적용)
final_pred_weighted = (oof_test_xgb * best_weight) + (oof_test_lgb * (1 - best_weight))

NameError: name 'np' is not defined

In [17]:
# %% [Step 1] OOF 다시 생성 (최신 파라미터 + 19개 정예 피처 적용)

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# 0. 최적 파라미터 가져오기 (방금 Optuna로 찾은 값)
final_params_xgb = best_params_xgb
final_params_lgb = best_params_lgb

print("✅ 적용된 XGB 파라미터:", final_params_xgb)
print("✅ 적용된 LGB 파라미터:", final_params_lgb)

# 1. OOF 생성 함수
def get_oof_predictions(model, X, y, X_test, n_splits=5, random_state=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    oof_train = np.zeros((X.shape[0], 3))
    oof_test_skf = np.zeros((X_test.shape[0], 3))
    
    print(f"Creating OOF for {model.__class__.__name__}...", end="")
    
    # DataFrame일 경우 iloc, numpy일 경우 인덱싱 처리
    X_values = X.values if hasattr(X, 'values') else X
    y_values = y
    X_test_values = X_test.values if hasattr(X_test, 'values') else X_test

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_values, y_values)):
        X_tr, y_tr = X_values[train_idx], y_values[train_idx]
        X_val = X_values[val_idx]
        
        clf = clone(model)
        clf.fit(X_tr, y_tr)
        
        oof_train[val_idx] = clf.predict_proba(X_val)
        oof_test_skf += clf.predict_proba(X_test_values) / n_splits
        print(f".", end="")
        
    print(" Done!")
    return oof_train, oof_test_skf

# 2. 모델 정의
xgb_model = XGBClassifier(**final_params_xgb)
lgb_model = LGBMClassifier(**final_params_lgb)

# 3. OOF 생성 실행 (X_xgb, X_lgb는 이미 19개 피처로 준비됨)
print("🚀 OOF 생성 시작 (XGBoost)...")
oof_train_xgb, oof_test_xgb = get_oof_predictions(xgb_model, X_xgb, y, X_xgb_test)

print("🚀 OOF 생성 시작 (LightGBM)...")
oof_train_lgb, oof_test_lgb = get_oof_predictions(lgb_model, X_lgb, y, X_lgb_test)

print("\n✅ OOF 생성 완료!")

✅ 적용된 XGB 파라미터: {'n_estimators': 788, 'learning_rate': 0.06238066751566709, 'max_depth': 6, 'min_child_weight': 1, 'gamma': 0.3960858015669083, 'subsample': 0.9214205395769955, 'colsample_bytree': 0.5491320112459366, 'reg_alpha': 0.002669920381142417, 'reg_lambda': 0.015444177908340267, 'objective': 'multi:softprob', 'eval_metric': 'mlogloss', 'tree_method': 'hist', 'random_state': 42, 'n_jobs': -1}
✅ 적용된 LGB 파라미터: {'n_estimators': 912, 'learning_rate': 0.01090933368427726, 'num_leaves': 193, 'max_depth': 14, 'min_child_samples': 51, 'subsample': 0.5933220031847126, 'colsample_bytree': 0.5036928973563994, 'reg_alpha': 0.008835424632453443, 'reg_lambda': 0.07034822683895961, 'objective': 'multiclass', 'random_state': 42, 'n_jobs': -1, 'verbosity': -1}
🚀 OOF 생성 시작 (XGBoost)...
Creating OOF for XGBClassifier........ Done!
🚀 OOF 생성 시작 (LightGBM)...
Creating OOF for LGBMClassifier........ Done!

✅ OOF 생성 완료!


In [18]:
# %% [Step 2] X_meta_train 생성 (Stacking)

# 1. 메타 모델용 데이터 만들기 (XGB 결과 + LGB 결과)
# (N, 3) + (N, 3) -> (N, 6) 형태의 데이터가 됨
X_meta_train = np.hstack([oof_train_xgb, oof_train_lgb])
X_meta_test = np.hstack([oof_test_xgb, oof_test_lgb])

print(f"✅ X_meta_train 생성 완료! Shape: {X_meta_train.shape}")
print(f"   (앞 3열: XGB 예측확률, 뒤 3열: LGB 예측확률)")

✅ X_meta_train 생성 완료! Shape: (6500, 6)
   (앞 3열: XGB 예측확률, 뒤 3열: LGB 예측확률)


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import optuna
from optuna.samplers import TPESampler
import pandas as pd

# 1. 평가 함수
def score_function_meta(y_true, y_pred_proba):
    if y_pred_proba.shape[1] > 2:
        y_bin = label_binarize(y_true, classes=np.unique(y_true))
        return average_precision_score(y_bin, y_pred_proba, average="macro")
    else:
        return average_precision_score(y_true, y_pred_proba[:, 1], average="macro")

# 2. Optuna 목적 함수
def objective_meta(trial):
    params = {
        'C': trial.suggest_float('C', 0.001, 100.0, log=True),
        'penalty': 'l2',
        'solver': 'lbfgs',
        'random_state': 42,
        'max_iter': 1000
    }
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in skf.split(X_meta_train, y):
        X_tr, X_val = X_meta_train[train_idx], X_meta_train[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        
        model = LogisticRegression(**params)
        model.fit(X_tr, y_tr)
        preds = model.predict_proba(X_val)
        scores.append(score_function_meta(y_val, preds))
    
    return np.mean(scores)

# 3. 튜닝 실행
print("🚀 메타 모델 튜닝 시작...")
study_meta = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_meta.optimize(objective_meta, n_trials=500, show_progress_bar=True)

print(f"🏆 Meta Best mAP: {study_meta.best_value:.6f}")
print("Best Params:", study_meta.best_params)

[I 2025-11-25 09:10:21,288] A new study created in memory with name: no-name-c4e5a003-03e6-47a9-91f9-deb70d668791


🚀 메타 모델 튜닝 시작...


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-11-25 09:10:21,422] Trial 0 finished with value: 0.6986764488717212 and parameters: {'C': 0.0745934328572655}. Best is trial 0 with value: 0.6986764488717212.
[I 2025-11-25 09:10:21,558] Trial 1 finished with value: 0.7008152678151809 and parameters: {'C': 56.69849511478853}. Best is trial 1 with value: 0.7008152678151809.
[I 2025-11-25 09:10:21,696] Trial 2 finished with value: 0.7015458285606752 and parameters: {'C': 4.5705630998014515}. Best is trial 2 with value: 0.7015458285606752.
[I 2025-11-25 09:10:21,796] Trial 3 finished with value: 0.7019486019384439 and parameters: {'C': 0.9846738873614566}. Best is trial 3 with value: 0.7019486019384439.
[I 2025-11-25 09:10:21,855] Trial 4 finished with value: 0.6734935441057419 and parameters: {'C': 0.006026889128682512}. Best is trial 3 with value: 0.7019486019384439.
[I 2025-11-25 09:10:21,921] Trial 5 finished with value: 0.6734864248466457 and parameters: {'C': 0.0060252157362038605}. Best is trial 3 with value: 0.701948601938

In [21]:
# %% [Final] 최종 제출 파일 생성 (Stacking & Weighted)

from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

# -----------------------------------------------------------------------------
# 1. 메타 모델(Stacking) 최종 학습 및 예측
# -----------------------------------------------------------------------------
print("🚀 메타 모델 최종 학습 중...")

# Optuna로 찾은 최적 파라미터 사용
best_meta_params = study_meta.best_params

# 메타 모델 정의 (최적 파라미터 적용)
final_meta_model = LogisticRegression(
    **best_meta_params,
    solver='lbfgs',
    random_state=42,
    max_iter=1000
)

# 전체 OOF 데이터(X_meta_train)로 학습
final_meta_model.fit(X_meta_train, y)

# 테스트 데이터(X_meta_test) 예측
final_pred_stacking = final_meta_model.predict_proba(X_meta_test)

# -----------------------------------------------------------------------------
# 2. 가중치 앙상블(Weighted) 최종 계산
# -----------------------------------------------------------------------------
print("🚀 가중치 앙상블 적용 중 (XGB: 0.59, LGB: 0.41)...")

# X_meta_test는 [XGB_pred, LGB_pred] 형태로 되어 있음 (각 3개 컬럼)
# 앞 3개 열: XGBoost 예측값
pred_xgb_final = X_meta_test[:, :3]
# 뒤 3개 열: LightGBM 예측값
pred_lgb_final = X_meta_test[:, 3:]

# 최적 가중치 적용
W_XGB = 0.59
W_LGB = 0.41

final_pred_weighted = (pred_xgb_final * W_XGB) + (pred_lgb_final * W_LGB)

# -----------------------------------------------------------------------------
# 3. CSV 파일 저장
# -----------------------------------------------------------------------------

# (1) Stacking 결과 저장
submission_stacking = pd.DataFrame({
    "index": test_df["index"],
    "prob_class_0": final_pred_stacking[:, 0],
    "prob_class_1": final_pred_stacking[:, 1],
    "prob_class_2": final_pred_stacking[:, 2]
})

save_path_stacking = r"C:\Users\abc01\OneDrive\바탕 화면\submission_stacking_final.csv"
submission_stacking.to_csv(save_path_stacking, index=False)
print(f"🎉 [Stacking] 저장 완료: {save_path_stacking}")

# (2) Weighted Ensemble 결과 저장
submission_weighted = pd.DataFrame({
    "index": test_df["index"],
    "prob_class_0": final_pred_weighted[:, 0],
    "prob_class_1": final_pred_weighted[:, 1],
    "prob_class_2": final_pred_weighted[:, 2]
})

save_path_weighted = r"C:\Users\abc01\OneDrive\바탕 화면\submission_weighted_final.csv"
submission_weighted.to_csv(save_path_weighted, index=False)
print(f"🎉 [Weighted] 저장 완료: {save_path_weighted}")

print("\n✅ 모든 작업 완료! 두 파일을 리더보드에 제출해보세요.")

🚀 메타 모델 최종 학습 중...
🚀 가중치 앙상블 적용 중 (XGB: 0.59, LGB: 0.41)...
🎉 [Stacking] 저장 완료: C:\Users\abc01\OneDrive\바탕 화면\submission_stacking_final.csv
🎉 [Weighted] 저장 완료: C:\Users\abc01\OneDrive\바탕 화면\submission_weighted_final.csv

✅ 모든 작업 완료! 두 파일을 리더보드에 제출해보세요.


In [23]:
# %% [Step 2-3] 랭크 앙상블 (Rank Ensemble)

from scipy.stats import rankdata

# 1. 확률 -> 랭크 변환 함수 (0~1 사이 값으로 정규화)
def get_rank_predictions(proba_array):
    # 각 클래스(컬럼)별로 순위를 매김
    ranks = np.zeros_like(proba_array)
    for i in range(proba_array.shape[1]):
        # rankdata: 작은 값부터 1등, 2등... (오름차순)
        # 정규화: 전체 개수로 나눠서 0~1 사이 값으로 맞춤
        ranks[:, i] = rankdata(proba_array[:, i]) / len(proba_array)
    return ranks

print("🚀 확률을 랭크(순위)로 변환 중...")

# Train OOF 랭크 변환
rank_train_xgb = get_rank_predictions(oof_train_xgb)
rank_train_lgb = get_rank_predictions(oof_train_lgb)

# Test 예측값 랭크 변환 (Test 데이터 기준)
# *주의: oof_test_xgb/lgb는 5-Fold 평균낸 Test 예측값입니다.
rank_test_xgb = get_rank_predictions(oof_test_xgb)
rank_test_lgb = get_rank_predictions(oof_test_lgb)


# 2. 랭크 기반 최적 가중치 찾기
best_score_rank = 0
best_weight_rank = 0

print("⚖️ 랭크 앙상블 최적 가중치 탐색 중...")

for w in np.arange(0.0, 1.01, 0.01):
    # 랭크끼리 가중치 적용해서 섞음
    blended_rank = (rank_train_xgb * w) + (rank_train_lgb * (1 - w))
    
    # mAP 점수 계산 (랭크값으로도 mAP 계산 가능)
    current_score = score_function(y, blended_rank)
    
    if current_score > best_score_rank:
        best_score_rank = current_score
        best_weight_rank = w

print(f"🏆 Rank Ensemble 최적 가중치! XGB: {best_weight_rank:.2f}, LGB: {1-best_weight_rank:.2f}")
print(f"   그때의 CV mAP (Rank): {best_score_rank:.6f}")

# 3. 최종 Test 예측 (랭크 앙상블 적용)
final_pred_rank = (rank_test_xgb * best_weight_rank) + (rank_test_lgb * (1 - best_weight_rank))


# 4. 제출 파일 저장
submission_rank = pd.DataFrame({
    "index": test_df["index"],
    "prob_class_0": final_pred_rank[:, 0],
    "prob_class_1": final_pred_rank[:, 1],
    "prob_class_2": final_pred_rank[:, 2]
})

save_path_rank = r"C:\Users\abc01\OneDrive\바탕 화면\submission_rank_final.csv"
submission_rank.to_csv(save_path_rank, index=False)
print(f"🎉 [Rank Ensemble] 저장 완료: {save_path_rank}")


🚀 확률을 랭크(순위)로 변환 중...
⚖️ 랭크 앙상블 최적 가중치 탐색 중...
🏆 Rank Ensemble 최적 가중치! XGB: 0.75, LGB: 0.25
   그때의 CV mAP (Rank): 0.704314
🎉 [Rank Ensemble] 저장 완료: C:\Users\abc01\OneDrive\바탕 화면\submission_rank_final.csv
